In [54]:
import cv2
import numpy as np
import os
import glob

In [55]:
def add_edge_false_color(image , shift_amount = 2):
    img_float = image.astype(np.float32)
    b , g , r = cv2.split(img_float)
    r_shifted = np.roll(r , shift_amount , axis=1)
    b_shifted = np.roll(b , -shift_amount , axis=1)
    merged = cv2.merge([b_shifted , g , r_shifted])
    merged = np.clip(merged , 0 , 255).astype(np.uint8)
    diff = cv2.absdiff(image , merged)
    diff_gray = cv2.cvtColor(diff , cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(diff_gray, 30, 255, cv2.THRESH_BINARY)

    return merged , mask



In [56]:
def add_random_edge_false_color(image, severity=2, num_patches=5):

    rows, cols = image.shape[:2]
    img_float = image.astype(np.float32)

    b, g, r = cv2.split(img_float)
    shift = np.random.randint(1, severity +1)

    r_shifted = np.roll(r, shift, axis=1)
    b_shifted = np.roll(b, -shift, axis=1)

    merged_defective = cv2.merge([b_shifted, g, r_shifted])
    merged_defective = np.clip(merged_defective, 0, 255).astype(np.uint8)

    selection_mask = np.zeros((rows, cols), dtype=np.float32)

    for _ in range(num_patches):
        size = np.random.randint(min(rows, cols) // 10, min(rows, cols) // 3)
        center_x = np.random.randint(0, cols)
        center_y = np.random.randint(0, rows)

        cv2.circle(selection_mask, (center_x, center_y), size, 1.0, -1)

    selection_mask = cv2.GaussianBlur(selection_mask, (21, 21), 0)

    selection_mask_3ch = cv2.merge([selection_mask, selection_mask, selection_mask])

    image_float = image.astype(np.float32)
    merged_defective_float = merged_defective.astype(np.float32)

    final_image = image_float * (1 - selection_mask_3ch) + merged_defective_float * selection_mask_3ch
    final_image = np.clip(final_image, 0, 255).astype(np.uint8)


    diff = cv2.absdiff(image, final_image)
    diff_gray = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)

    _, gt_mask = cv2.threshold(diff_gray, 20, 255, cv2.THRESH_BINARY)

    return final_image, gt_mask

In [57]:
def process_folder(input_folder_path, output_base_path):

    images_output_dir = os.path.join(output_base_path, "defective_images")
    masks_output_dir = os.path.join(output_base_path, "masks")

    images_output_random_dir = os.path.join(output_base_path, "random_defective_images")
    masks_output_random_dir = os.path.join(output_base_path, "random_masks")

    os.makedirs(images_output_dir, exist_ok=True)
    os.makedirs(masks_output_dir, exist_ok=True)
    os.makedirs(images_output_random_dir, exist_ok=True)
    os.makedirs(masks_output_random_dir, exist_ok=True)

    extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
    image_files = []
    for ext in extensions:
        image_files.extend(glob.glob(os.path.join(input_folder_path, ext)))

    print(f"Found {len(image_files)} images. Starting processing...")

    for file_path in image_files:
        img = cv2.imread(file_path)
        if img is None:
            print(f"Could not read {file_path}. Skipping.")
            continue

        filename = os.path.basename(file_path)


        defective_img, mask = add_edge_false_color(img)


        defective_img_random, mask_random = add_random_edge_false_color(img)


        cv2.imwrite(os.path.join(images_output_dir, filename), defective_img)
        cv2.imwrite(os.path.join(masks_output_dir, filename), mask)


        cv2.imwrite(os.path.join(images_output_random_dir, filename), defective_img_random)
        cv2.imwrite(os.path.join(masks_output_random_dir, filename), mask_random)

    print("Done!")
    print(f"Full defects saved in: {images_output_dir}")
    print(f"Random defects saved in: {images_output_random_dir}")

In [58]:
print("---------- start of edge false color ----------")

SOURCE_DIR ="D:\dataset\Samsung\Original photos"
OUTPUT_BASE_DIR = "D:\dataset\Samsung\edge false color"
process_folder(SOURCE_DIR, OUTPUT_BASE_DIR)

print("---------- end of edge false color ----------")


---------- start of edge false color ----------
Found 2745 images. Starting processing...
Done!
Full defects saved in: D:\dataset\Samsung\edge false color\defective_images
Random defects saved in: D:\dataset\Samsung\edge false color\random_defective_images
---------- end of edge false color ----------
